In [1]:
# set run environment (local/colab)
# if colab install required packages and set appropriate root_path
import os

if os.getenv("COLAB_RELEASE_TAG"):
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = '/content/drive/Othercomputers/My computer/EQILLM/'
    !pip install -r '/content/drive/Othercomputers/My computer/EQILLM/requirements.txt'
    !pip install transformers[torch]
    !pip install accelerate -U
    !pip install datasets
    !pip install torchinfo
    import sys
    sys.path.append(root_path)
    #ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U
else:
    colab = False
    root_path = ''


import itertools
import pandas as pd
import openai
import datetime
import os
import csv

from dotenv import load_dotenv, dotenv_values
from tqdm.notebook import tqdm_notebook
from eqillm import finetune, get_log_for_val, validate, val_metrics, yeelight_eow_notification, param_combinations, load_PolarIs, df_to_ds


dotenv_config = dotenv_values(os.path.join(root_path, '.env'))
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if 'YEELIGHT_NOTIFY' in dotenv_config else False

In [14]:
data_path = os.path.join(root_path, 'data/PolarIs-Pathos.xlsx')
polarIs_df = load_PolarIs(data_path)

dataset, target_map, reversed_target_map = df_to_ds(polarIs_df, split=(0.1, 0.1, 0.8), binary=True, balanced=True)
val_txts, val_labels = dataset['train']['text'], [reversed_target_map[i] for i in dataset['train']['label']]
# val_txts, val_labels = polarIs_df['text'], polarIs_df['label']

C:\Users\Jakub\DataspellProjects\EQILLM\eqillm.py:104: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  train_test = [pd.concat([i[0], i[1]], axis=0) for i in zip(train_test, train_test_group)]


In [15]:
print(len(val_labels))

990


In [16]:
# [[i,q] for i,q in enumerate(val_txts) if 'Unlike the Occupy crowd' in q]
# print(*zip(val_txts[587:] , val_labels[587:]), sep='\n')
# val_txts , val_labels = val_txts[588:] , val_labels[588:]

In [17]:
def create_messages(text='_'):
    return [
        {
            'role': 'system',
             'content': "You are an analytical tool designed on identifying pathos in Text. Pathos is an appeal made to an audience’s emotions in order to evoke feeling. Your task is to analyze if the speaker's intention is to elicit specific emotions for rhetorical purposes. If so, respond with 'Pathos'. Otherwise respond with 'No Pathos'"
        },
        {
            "role": "user",
            "content": f"""Analyze the following text for pathos: {text}"""
        },
        {
            "role": "assistant",
            "content": "Based on the analysis, is there an attempt to elicit an emotion? If so, respond with 'Pathos'. If not, respond with 'No pathos'. Do not identify emotions not listed."
        }
    ]

def looped_prompt(texts, labels, model_url=dotenv_config['MODEL_URL'], api_key=dotenv_config['MODEL_API_KEY']):
    client = openai.OpenAI(base_url=model_url,  api_key=api_key)
    model_name = client.models.list().data[0].dict()['id']

    prompt_log_path = os.path.join(root_path, 'output', 'prompting', 'prompt_log.csv')
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    response_log_path = os.path.join(root_path, 'output', 'prompting', 'responses', f'{timestamp}.csv')

    with open(prompt_log_path, mode='a+', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if not os.path.exists(prompt_log_path):
            writer.writerow(['model', 'timestamp', 'message_system', 'message_user', 'message_assistant'])
        writer.writerow([model_name, timestamp, *create_messages()])

    with open(response_log_path, mode='a+', newline='', encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file,  delimiter='|')
        writer.writerow(['text', 'label', 'timestamp', 'response'])

        for i, (txt, label) in tqdm_notebook(enumerate(zip(texts, labels))): # iterate through each text value of first df column
            completion = client.chat.completions.create(
                model="local-model", # this field is currently unused
                messages=create_messages(txt),
                temperature=0.5,
                max_tokens=50
            )
            writer.writerow([txt, label, timestamp, completion.choices[0].message.content])

In [18]:
looped_prompt(val_txts, val_labels)

if yeelight_notify:
    yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])

0it [00:00, ?it/s]